In [ ]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub  # For pre-trained models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as img
import PIL.Image as Image
import cv2
import os
import numpy as np
import pathlib

# Define class labels
class_labels = {
    'arborio': 0,
    'basmati': 1,
    'ipsala': 2,
    'jasmine': 3,
    'karacadag': 4
}

# File paths of uploaded images
image_paths = {
    'arborio': "/mnt/data/Arborio (999).jpg",
    'basmati': "/mnt/data/basmati (10028).jpg",
    'ipsala': "/mnt/data/Ipsala (9993).jpg",
    'jasmine': "/mnt/data/Jasmine (9997).jpg",
    'karacadag': "/mnt/data/Karacadag (998).jpg"
}

# Initialize lists for images and labels
X, y = [], []

# Preprocess each image
for label, path in image_paths.items():
    img = cv2.imread(path)
    if img is not None:
        # Convert to RGB format (since OpenCV loads as BGR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Resize image to (224, 224)
        img_resized = cv2.resize(img, (224, 224))
        img_resized = img_resized / 255.0  # Normalize
        X.append(img_resized)
        y.append(class_labels[label])

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# One-hot encode labels
y = keras.utils.to_categorical(y, num_classes=len(class_labels))

# Standardizing images
X = np.array(X) / 255.0  # Normalizing pixel values
y = np.array(y)  # Converting labels to numpy array

# Splitting into Training (70%) and Test + Validation (30%)
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Splitting Test + Validation into separate Validation (15%) and Test (15%)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42, stratify=y_test_val)

# Display dataset sizes
print(f"Training Set: {X_train.shape}, Labels: {y_train.shape}")
print(f"Validation Set: {X_val.shape}, Labels: {y_val.shape}")
print(f"Test Set: {X_test.shape}, Labels: {y_test.shape}")

# ---- Preview Images ----
fig, axes = plt.subplots(1, 5, figsize=(20, 5))
fig.suptitle("Rice Categories")

# Extract sample images from the training set
sample_labels = ["Arborio", "Basmati", "Ipsala", "Jasmine", "Karacadag"]
for i, ax in enumerate(axes):
    ax.imshow(X_train[i])
    ax.set_title(sample_labels[np.argmax(y_train[i])])
    ax.axis("off")

plt.show()

# ---- Model Definition ----
mobilenet_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
mobile_net = hub.KerasLayer(mobilenet_url, input_shape=(224, 224, 3), trainable=False)

num_labels = 5  # Number of classes

model = keras.Sequential([
    mobile_net,
    keras.layers.Dense(num_labels, activation='softmax')
])

# Model summary
model.summary()

# Compile the model
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

# ---- Training the Model ----
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=8
)

# ---- Evaluating the Model ----
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# ---- Plot Training Performance ----
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Accuracy Over Epochs")

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Loss Over Epochs")
plt.show()
